In [1]:
import graphviz

# Crear el objeto del grafo
dot = graphviz.Digraph('flujo_main', comment='Diagrama de Flujo de main.py')
dot.attr(rankdir='TB', splines='ortho', label='Diagrama de Flujo Lógico de main.py', fontsize='20')

# Definir estilos para los nodos
action_style = {'shape': 'box', 'style': 'rounded,filled', 'fillcolor': 'lightblue'}
decision_style = {'shape': 'diamond', 'style': 'filled', 'fillcolor': 'lightyellow'}
io_style = {'shape': 'parallelogram', 'style': 'filled', 'fillcolor': 'lightgrey'}
start_end_style = {'shape': 'ellipse', 'style': 'filled', 'fillcolor': 'palegreen'}

# 1. Nodos del Flujo
# --- Setup ---
dot.node('start', 'Inicio', **start_end_style)
dot.node('setup', 'Configuración Inicial\n(Cargar Modelo, Conectar DB/Arduino, Iniciar Cámara)', **action_style)

# --- Bucle Principal ---
dot.node('loop_start', 'Bucle Principal (while True)', shape='box', style='dashed')
dot.node('captura', 'Capturar Frame de Cámara', **io_style)
dot.node('yolo', 'Detectar Placa (YOLO)', **action_style)
dot.node('check_detection', '¿Placa Detectada?', **decision_style)

# --- Lógica de Estabilización ---
dot.node('start_timer', 'Iniciar Temporizador (3s)', **action_style)
dot.node('check_timer', '¿Pasaron 3 seg?', **decision_style)
dot.node('update_roi', 'Actualizar ROI Estabilizada', **action_style)

# --- Procesamiento OCR ---
dot.node('process_roi', 'Preprocesar ROI y Reconocer Texto (OCR)', **action_style)
dot.node('check_ocr', '¿Texto Reconocido?', **decision_style)

# --- Lógica de Negocio ---
dot.node('check_db', 'Verificar Placa en DB', **action_style)
dot.node('check_auth', '¿Placa Registrada?', **decision_style)
dot.node('open_gate', 'Abrir Barrera y Guardar Registro', **action_style)
dot.node('show_gui', 'Mostrar GUI de Registro', **action_style)

# --- Finalización y Reintentos ---
dot.node('reset_success', 'Reiniciar Detección (Pausa 5s)', **action_style)
dot.node('reset_fail', 'Reiniciar Detección (Pausa 2s)', **action_style)
dot.node('check_quit', '¿Usuario presiona "q"?', **decision_style)
dot.node('end', 'Finalizar\n(Liberar Cámara, Cerrar Conexiones)', **start_end_style)


# 2. Conexiones del Flujo
dot.edge('start', 'setup')
dot.edge('setup', 'loop_start')
dot.edge('loop_start', 'captura')
dot.edge('captura', 'yolo')
dot.edge('yolo', 'check_detection')

# Flujo si no se detecta placa
dot.edge('check_detection', 'check_quit', label='No')

# Flujo si se detecta placa
dot.edge('check_detection', 'start_timer', label='Sí (Primera vez)')
dot.edge('start_timer', 'check_timer')
dot.edge('check_detection', 'check_timer', label='Sí (Ya detectada)')

# Flujo del temporizador
dot.edge('check_timer', 'update_roi', label='No (< 3s)')
dot.edge('update_roi', 'check_quit')
dot.edge('check_timer', 'process_roi', label='Sí (>= 3s)')

# Flujo del OCR
dot.edge('process_roi', 'check_ocr')
dot.edge('check_ocr', 'check_db', label='Sí')
dot.edge('check_ocr', 'reset_fail', label='No')

# Flujo de la Base de Datos
dot.edge('check_db', 'check_auth')
dot.edge('check_auth', 'open_gate', label='Sí (Autorizada)')
dot.edge('check_auth', 'show_gui', label='No (No Registrada)')
dot.edge('open_gate', 'reset_success')
dot.edge('show_gui', 'reset_success')

# Flujos de reinicio y salida
dot.edge('reset_success', 'check_quit')
dot.edge('reset_fail', 'check_quit')
dot.edge('check_quit', 'loop_start', label='No')
dot.edge('check_quit', 'end', label='Sí')


# 3. Generar y guardar el diagrama
try:
    dot.render('flujo_main', format='pdf', view=True, cleanup=True)
    print("\n✅ ¡Diagrama 'flujo_main.pdf' generado con éxito!")
except graphviz.backend.execute.ExecutableNotFound:
    print("\n❌ Error: Graphviz no está instalado en tu sistema.")
    print("Recuerda ejecutar: sudo apt-get install graphviz")


✅ ¡Diagrama 'flujo_main.pdf' generado con éxito!


In [2]:
import graphviz

dot = graphviz.Digraph('arquitectura_sistema_ocr_v2', comment='Diagrama de Flujo con Sensor de Proximidad')
dot.attr(rankdir='TB', splines='ortho', label='Arquitectura con Control de Proximidad', fontsize='20')

# Definir estilos
action_style = {'shape': 'box', 'style': 'rounded,filled', 'fillcolor': 'lightblue'}
decision_style = {'shape': 'diamond', 'style': 'filled', 'fillcolor': 'lightyellow'}
io_style = {'shape': 'parallelogram', 'style': 'filled', 'fillcolor': 'lightgrey'}
start_end_style = {'shape': 'ellipse', 'style': 'filled', 'fillcolor': 'palegreen'}
control_style = {'shape': 'box', 'style': 'rounded,filled', 'fillcolor': '#a2d2ff'} # Color para el bloque de control

# Nodos
with dot.subgraph(name='cluster_hardware') as c:
    c.attr(style='filled', color='lightgrey', label='Hardware y Sensores')
    c.node('prox_sensor', '➡️ Sensor de Proximidad', shape='box') # <-- NODO NUEVO
    c.node('camara', '📷 Cámara', shape='box')
    c.node('arduino', '🤖 Controlador Arduino', shape='box')
    c.node('flash', '💡 Flash / Iluminador', shape='box')
    c.node('talanquera', '🚧 Talanquera / Barrera', shape='box')

with dot.subgraph(name='cluster_software') as c:
    c.attr(label='Procesamiento de Software (Python)')
    c.node('listener', 'Esperar Señal de "CARRO_DETECTADO"', **io_style) # <-- LÓGICA NUEVA
    c.node('ocr_pipeline', 'Pipeline de OCR\n(Captura -> YOLO -> Preproc -> Tesseract)', **action_style)
    c.node('db_logic', 'Lógica de Base de Datos\n(Verificar/Registrar Placa)', **action_style)
    c.node('db', 'Base de Datos (SQL Server)', shape='cylinder')

with dot.subgraph(name='cluster_control') as c:
    c.attr(label='Control de Luminosidad (Opcional/Avanzado)', style='filled', color='#bde0fe')
    c.node('fuzzy_control', 'Controlador de Luminosidad (Fuzzy/RL)')


# Conexiones
dot.edge('prox_sensor', 'arduino')
dot.edge('arduino', 'listener', label='Señal "CARRO_DETECTADO"') # <-- DISPARADOR PRINCIPAL

# El bucle de procesamiento solo se activa después de la señal
dot.edge('listener', 'ocr_pipeline', label='Activar')
dot.edge('ocr_pipeline', 'db_logic', label='Texto de Placa')
dot.edge('db_logic', 'db')

# Salidas
dot.edge('db_logic', 'arduino', label='Comando "OPEN"')
dot.edge('arduino', 'talanquera')

# Bucle de control de luz (ahora parte del pipeline)
dot.edge('ocr_pipeline', 'fuzzy_control', label='Confianza OCR')
dot.edge('fuzzy_control', 'arduino', label='Ajuste de Flash')
dot.edge('arduino', 'flash')

# Regreso al estado de espera
dot.edge('db_logic', 'listener', label='Resetear y Esperar\nSiguiente Vehículo', style='dashed')


# Generar y guardar el diagrama
try:
    dot.render('arquitectura_con_proximidad', format='pdf', view=True, cleanup=True)
    print("\n✅ ¡Diagrama 'arquitectura_con_proximidad.pdf' generado con éxito!")
except Exception as e:
    print(f"\n❌ Error al generar el diagrama: {e}")
    print("Asegúrate de que Graphviz esté instalado en tu sistema.")


✅ ¡Diagrama 'arquitectura_con_proximidad.pdf' generado con éxito!


In [1]:
import os

print("Iniciando la creación de los archivos del proyecto...")

# --- 1. Creando arduino_python.py ---
arduino_code = """
import serial
import time
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

class FuzzyController:
    def __init__(self):
        self.luz_ambiental = ctrl.Antecedent(np.arange(0, 101, 1), 'luz_ambiental')
        self.confianza_ocr = ctrl.Antecedent(np.arange(0, 101, 1), 'confianza_ocr')
        self.potencia_flash = ctrl.Consequent(np.arange(0, 256, 1), 'potencia_flash')
        # Se definen valores iniciales, pero el agente de RL los ajustará
        self.tune([50, 80, 40, 70])

    def tune(self, params):
        '''
        Ajusta las funciones de membresía del controlador difuso.
        Esta es la 'ACCIÓN' que el agente de RL podrá tomar.
        '''
        conf_media_fin, conf_alta_inicio, luz_media_fin, luz_clara_inicio = params
        self.confianza_ocr['baja'] = fuzz.trimf(self.confianza_ocr.universe, [0, 0, conf_media_fin])
        self.confianza_ocr['media'] = fuzz.trimf(self.confianza_ocr.universe, [0, conf_media_fin, conf_alta_inicio])
        self.confianza_ocr['alta'] = fuzz.trimf(self.confianza_ocr.universe, [conf_alta_inicio, 100, 100])
        self.luz_ambiental['oscura'] = fuzz.trimf(self.luz_ambiental.universe, [0, 0, luz_media_fin])
        self.luz_ambiental['media'] = fuzz.trimf(self.luz_ambiental.universe, [0, luz_media_fin, luz_clara_inicio])
        self.luz_ambiental['clara'] = fuzz.trimf(self.luz_ambiental.universe, [luz_clara_inicio, 100, 100])
        self.potencia_flash['baja'] = fuzz.trimf(self.potencia_flash.universe, [0, 0, 85])
        self.potencia_flash['media'] = fuzz.trimf(self.potencia_flash.universe, [80, 170, 255])
        self.potencia_flash['alta'] = fuzz.trimf(self.potencia_flash.universe, [200, 255, 255])
        
        rule1 = ctrl.Rule(self.confianza_ocr['baja'], self.potencia_flash['alta'])
        rule2 = ctrl.Rule(self.confianza_ocr['media'] & self.luz_ambiental['oscura'], self.potencia_flash['media'])
        rule3 = ctrl.Rule(self.confianza_ocr['alta'] | self.luz_ambiental['clara'], self.potencia_flash['baja'])
        
        self.control_system = ctrl.ControlSystem([rule1, rule2, rule3])
        self.simulation = ctrl.ControlSystemSimulation(self.control_system)

    def compute(self, input_luz, input_confianza):
        self.simulation.input['luz_ambiental'] = input_luz
        self.simulation.input['confianza_ocr'] = input_confianza
        try:
            self.simulation.compute()
            return self.simulation.output['potencia_flash']
        except:
            return 50.0

def conectar_arduino(port='COM4', baudrate=9600):
    puertos_linux = ['/dev/ttyUSB0', '/dev/ttyACM0']
    try:
        arduino = serial.Serial(port, baudrate, timeout=1)
        print(f"Conexión con Arduino establecida en {port}.")
        return arduino
    except serial.SerialException:
        for p in puertos_linux:
            try:
                arduino = serial.Serial(p, baudrate, timeout=1)
                print(f"Conexión con Arduino establecida en {p}.")
                return arduino
            except serial.SerialException:
                continue
        print(f"Error: No se pudo conectar con Arduino.")
        return None

def enviar_comando_arduino(arduino, comando):
    if arduino and arduino.is_open:
        arduino.write(f"{comando}\\n".encode())
        print(f"Comando enviado al Arduino: {comando}")

def cerrar_conexion_arduino(arduino):
    if arduino and arduino.is_open:
        arduino.close()
        print("Conexión con Arduino cerrada.")
"""
with open("arduino_python.py", "w") as f:
    f.write(arduino_code)
print("✅ Archivo 'arduino_python.py' creado.")


# --- 2. Creando database_sql.py ---
database_code = """
import pyodbc
import tkinter as tk
from tkinter import messagebox

def conectar_sql_server():
    try:
        conn = pyodbc.connect(
            r"Driver={SQL Server};"
            r"Server=DESKTOP-KMG57VB\\SQLEXPRESS;"
            r"Database=Registro_Placas;"
            r"Trusted_Connection=yes;"
        )
        print("Conexión con SQL Server establecida.")
        return conn
    except Exception as e:
        print(f"Error al conectar con SQL Server: {e}")  
        return None

def guardar_en_base_de_datos(conn, placa):
    try:
        cursor = conn.cursor()
        query = "INSERT INTO PlacasDetectadas (Placa) VALUES (?)"
        cursor.execute(query, placa)
        conn.commit()
        print(f"Placa '{placa}' registrada en la base de datos.")
    except Exception as e:
        print(f"Error al guardar en la base de datos: {e}")

def verificar_placa_registrada(conn, placa):
    try:
        cursor = conn.cursor()
        query = "SELECT * FROM VehiculosRegistrados WHERE Placa = ?"
        cursor.execute(query, placa)
        return cursor.fetchone()
    except Exception as e:
        print(f"Error al verificar la placa: {e}")
        return None

def mostrar_interfaz_registro(conn, placa):
    def registrar():
        nombre = entry_nombre.get()
        marca = entry_marca.get()
        modelo = entry_modelo.get()
        color = entry_color.get()

        if not all([nombre, marca, modelo, color]):
            messagebox.showerror("Error", "Por favor, completa todos los campos.")
            return

        try:
            cursor = conn.cursor()
            query = "INSERT INTO VehiculosRegistrados (Placa, NombreCompleto, Marca, Modelo, Color) VALUES (?, ?, ?, ?, ?)"
            cursor.execute(query, (placa, nombre, marca, modelo, color))
            conn.commit()
            messagebox.showinfo("Éxito", f"Vehículo con placa '{placa}' registrado exitosamente.")
            ventana.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"Error al registrar el vehículo: {e}")

    ventana = tk.Tk()
    ventana.title("Registro de Vehículo")
    ventana.geometry("400x300")
    tk.Label(ventana, text=f"Placa detectada: {placa}", font=("Arial", 14)).pack(pady=10)
    
    tk.Label(ventana, text="Nombre del dueño:").pack()
    entry_nombre = tk.Entry(ventana, width=30)
    entry_nombre.pack()
    tk.Label(ventana, text="Marca del vehículo:").pack()
    entry_marca = tk.Entry(ventana, width=30)
    entry_marca.pack()
    tk.Label(ventana, text="Modelo del vehículo:").pack()
    entry_modelo = tk.Entry(ventana, width=30)
    entry_modelo.pack()
    tk.Label(ventana, text="Color del vehículo:").pack()
    entry_color = tk.Entry(ventana, width=30)
    entry_color.pack()
    
    tk.Button(ventana, text="Registrar", command=registrar).pack(pady=20)
    ventana.mainloop()
"""
with open("database_sql.py", "w") as f:
    f.write(database_code)
print("✅ Archivo 'database_sql.py' creado.")


# --- 3. Creando entrenamiento.py ---
entrenamiento_yolo_code = """
from ultralytics import YOLO

# Carga un modelo base (ej. yolov8n.pt)
model = YOLO('yolov8n.pt')

# Entrena el modelo con tu dataset
results = model.train(
    data='placas.v1i.yolov8/data.yaml',
    epochs=30,
    imgsz=320,
    batch=4,
    name='placas_v14',
    device='cpu' # Cambia a 'cuda' si tienes una GPU configurada
)

print("Entrenamiento de YOLO finalizado.")
"""
with open("entrenamiento.py", "w") as f:
    f.write(entrenamiento_yolo_code)
print("✅ Archivo 'entrenamiento.py' creado.")


# --- 4. Creando main.py ---
main_app_code = """
import cv2
import time
from ultralytics import YOLO
from arduino_python import conectar_arduino, enviar_comando_arduino, cerrar_conexion_arduino
from database_sql import conectar_sql_server, guardar_en_base_de_datos, verificar_placa_registrada, mostrar_interfaz_registro
from preprocesamiento import preprocesar_placa, detectar_texto

# --- SETUP ---
model = YOLO("runs/detect/placas_v14/weights/best.pt")
arduino = conectar_arduino()
conn = conectar_sql_server()

if conn is None:
    print("❌ No se pudo establecer conexión con SQL Server. Finalizando...")
    exit()

cap = cv2.VideoCapture(0)
placa_detectada = False
start_time = None
roi_estabilizada = None

print("Iniciando detección en tiempo real...")

# --- LOOP ---
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, imgsz=320, conf=0.5)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            roi = frame[y1:y2, x1:x2]

            if not placa_detectada:
                print("Placa detectada. Esperando estabilización...")
                start_time = time.time()
                roi_estabilizada = roi.copy()
                placa_detectada = True
            
            elif time.time() - start_time < 3:
                roi_estabilizada = roi.copy()

            elif time.time() - start_time >= 3:
                print("Capturando imagen y reconociendo caracteres...")
                img_preprocesada = preprocesar_placa(roi_estabilizada)
                if img_preprocesada is not None:
                    # La función detectar_texto ahora devuelve texto y confianza
                    placa_texto, confianza = detectar_texto(img_preprocesada)
                    
                    if placa_texto:
                        print(f"Placa reconocida: {placa_texto} (Confianza: {confianza:.2f}%)")
                        vehiculo = verificar_placa_registrada(conn, placa_texto)
                        
                        if vehiculo:
                            print(f"Placa autorizada. Vehículo: {vehiculo}")
                            guardar_en_base_de_datos(conn, placa_texto)
                            enviar_comando_arduino(arduino, "OPEN")
                        else:
                            print("Placa no registrada. Solicitar registro.")
                            mostrar_interfaz_registro(conn, placa_texto)
                        
                        placa_detectada = False
                        time.sleep(5)
                    else:
                        print("No se reconoció texto en la placa. Reintentando...")
                        placa_detectada = False
                        time.sleep(2)

    cv2.imshow("📷 Detección de placas", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- CLEANUP ---
cap.release()
cv2.destroyAllWindows()
cerrar_conexion_arduino(arduino)
if conn:
    conn.close()
"""
with open("main.py", "w") as f:
    f.write(main_app_code)
print("✅ Archivo 'main.py' creado.")


# --- 5. Creando preprocesamiento.py ---
preprocesamiento_code = """
import cv2
import pytesseract
import pandas as pd
import re

# Asegurarse de que Python sepa dónde encontrar Tesseract en Ubuntu
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

def preprocesar_placa(roi):
    try:
        gris = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gauss = cv2.GaussianBlur(gris, (5, 5), 0)
        umbral = cv2.adaptiveThreshold(
            gauss, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
        )
        return umbral
    except Exception as e:
        print(f"Error durante el preprocesamiento de la placa: {e}")
        return None

def detectar_texto(img_preprocesada):
    try:
        config = "--psm 8 --oem 3"
        data = pytesseract.image_to_data(img_preprocesada, config=config, output_type=pytesseract.Output.DATAFRAME)
        data = data[data.conf > 0]
        
        if not data.empty:
            texto_detectado = "".join(data['text'].astype(str))
            texto_normalizado = re.sub(r'[^A-Z0-9]', '', texto_detectado.upper())
            confianza_promedio = data['conf'].mean()
            return texto_normalizado, confianza_promedio
        else:
            return None, 0
    except Exception as e:
        print(f"Error durante la detección de texto con Tesseract: {e}")
        return None, 0
"""
with open("preprocesamiento.py", "w") as f:
    f.write(preprocesamiento_code)
print("✅ Archivo 'preprocesamiento.py' creado.")


# --- 6. Creando simulation_manager.py ---
simulation_manager_code = """
import cv2
import numpy as np
import os
import random

class SimulationManager:
    def __init__(self, environments_base_path):
        self.base_path = environments_base_path
        self.available_environments = [d for d in os.listdir(self.base_path) 
                                       if os.path.isdir(os.path.join(self.base_path, d))]
        if not self.available_environments:
            raise FileNotFoundError(f"No se encontraron directorios de entornos en '{self.base_path}'")
        print(f"✅ Gestor de Simulación inicializado. {len(self.available_environments)} entornos encontrados.")
        self.current_env_name = None
        self.images = {}
        self.sorted_images_by_intensity = []

    def load_environment(self, env_name):
        if env_name not in self.available_environments:
            raise ValueError(f"El entorno '{env_name}' no existe.")
        self.current_env_name = env_name
        env_path = os.path.join(self.base_path, env_name)
        self.images = {}
        for filename in os.listdir(env_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                path = os.path.join(env_path, filename)
                img = cv2.imread(path)
                if img is not None:
                    self.images[filename] = img
        if len(self.images) < 2:
            raise ValueError(f"El entorno '{env_name}' debe contener al menos 2 imágenes.")
        self.sorted_images_by_intensity = sorted(self.images.values(), key=lambda img: np.mean(cv2.cvtColor(img, cv2.COLOR_BGR2HSV)[:, :, 2]))

    def get_random_environment_name(self):
        return random.choice(self.available_environments)

    def simulate_lighting(self, intensity, activation_threshold=0.25):
        if not self.current_env_name:
            raise RuntimeError("Carga un entorno con 'load_environment()' primero.")
        intensity = np.clip(float(intensity), 0.0, 1.0)
        darkest_img = self.sorted_images_by_intensity[0]
        headlight_img = self.sorted_images_by_intensity[1]
        if intensity <= activation_threshold:
            base_dinamica = darkest_img.astype(np.float32)
        else:
            mix_factor = (intensity - activation_threshold) / (1.0 - activation_threshold)
            base_dinamica = cv2.addWeighted(darkest_img, 1.0 - mix_factor, headlight_img, mix_factor, 0).astype(np.float32)
        beta = (intensity - 0.5) * 50
        alpha = 1.0 + (intensity * 0.2)
        return cv2.convertScaleAbs(base_dinamica, alpha=alpha, beta=beta)
"""
with open("simulation_manager.py", "w") as f:
    f.write(simulation_manager_code)
print("✅ Archivo 'simulation_manager.py' creado.")


# --- 7. Creando ocr_environment.py ---
ocr_environment_code = """
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from simulation_manager import SimulationManager
from arduino_python import FuzzyController 
from preprocesamiento import preprocesar_placa, detectar_texto
from ultralytics import YOLO

class OCREnvironment(gym.Env):
    def __init__(self):
        super(OCREnvironment, self).__init__()
        self.sim_manager = SimulationManager(environments_base_path='entornos/')
        self.fuzzy_controller = FuzzyController()
        self.yolo_model = YOLO("runs/detect/placas_v14/weights/best.pt")
        self.action_space = spaces.Box(low=np.array([10, 51, 10, 51]), high=np.array([50, 90, 50, 90]), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=100, shape=(2,), dtype=np.float32)

    def step(self, action):
        self.fuzzy_controller.tune(action)
        ambient_light = np.random.uniform(5, 70)
        initial_confidence = 10 
        flash_power = self.fuzzy_controller.compute(ambient_light, initial_confidence)
        intensity = flash_power / 255.0
        simulated_image = self.sim_manager.simulate_lighting(intensity)
        
        ocr_results = self.yolo_model(simulated_image, imgsz=320, conf=0.5, verbose=False)
        final_confidence = 0.0
        if ocr_results and len(ocr_results[0].boxes) > 0:
            box = ocr_results[0].boxes[0]
            roi = simulated_image[int(box.xyxy[0][1]):int(box.xyxy[0][3]), int(box.xyxy[0][0]):int(box.xyxy[0][2])]
            img_preprocesada = preprocesar_placa(roi)
            if img_preprocesada is not None:
                _, confidence = detectar_texto(img_preprocesada)
                final_confidence = confidence if confidence is not None else 0.0

        reward = final_confidence
        terminated = True
        truncated = False
        observation = np.array([ambient_light, final_confidence], dtype=np.float32)
        info = {}
        return observation, reward, terminated, truncated, info

    def reset(self, seed=None):
        if seed is not None:
            super().reset(seed=seed)
        random_env = self.sim_manager.get_random_environment_name()
        self.sim_manager.load_environment(random_env)
        observation = np.array([50, 0], dtype=np.float32)
        info = {}
        return observation, info
"""
with open("ocr_environment.py", "w") as f:
    f.write(ocr_environment_code)
print("✅ Archivo 'ocr_environment.py' creado.")


# --- 8. Creando train_agent.py ---
train_agent_code = """
from stable_baselines3 import PPO
from ocr_environment import OCREnvironment
import torch

# Verificar si CUDA está disponible y establecer el dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

# 1. Crear el entorno personalizado
env = OCREnvironment()

# 2. Crear el agente PPO
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log="./ppo_tensorboard/", device=device)

print("\\n🚀 Iniciando entrenamiento del agente PPO...")

# 3. Entrenar el agente
model.learn(total_timesteps=10000)

print("\\n✅ Entrenamiento finalizado.")

# 4. Guardar el agente entrenado
model.save("ppo_fuzzy_ocr_controller")
print("\\n💾 Agente guardado como 'ppo_fuzzy_ocr_controller.zip'")
"""
with open("train_agent.py", "w") as f:
    f.write(train_agent_code)
print("✅ Archivo 'train_agent.py' creado.")

print("\n--- ¡Proceso completado! Todos los archivos .py han sido generados. ---")

Iniciando la creación de los archivos del proyecto...
✅ Archivo 'arduino_python.py' creado.
✅ Archivo 'database_sql.py' creado.
✅ Archivo 'entrenamiento.py' creado.
✅ Archivo 'main.py' creado.
✅ Archivo 'preprocesamiento.py' creado.
✅ Archivo 'simulation_manager.py' creado.
✅ Archivo 'ocr_environment.py' creado.
✅ Archivo 'train_agent.py' creado.

--- ¡Proceso completado! Todos los archivos .py han sido generados. ---


In [1]:
import os

print("Generando la versión final de todos los archivos del proyecto...")

# --- Contenido de arduino_python.py ---
arduino_python_code = """
import serial
import time
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import cv2

# --- VARIABLES GLOBALES PARA LA SIMULACIÓN ---
SIMULATION_MODE = False
dashboard = None
log_messages = ["Presiona 'D' para simular deteccion, 'Q' para salir."]
barrier_angle, target_angle = 0, 0
flash_brightness = 0

# --- CLASE DEL CONTROLADOR DIFUSO ---
class FuzzyController:
    def __init__(self):
        self.luz_ambiental = ctrl.Antecedent(np.arange(0, 101, 1), 'luz_ambiental')
        self.confianza_ocr = ctrl.Antecedent(np.arange(0, 101, 1), 'confianza_ocr')
        self.potencia_flash = ctrl.Consequent(np.arange(0, 256, 1), 'potencia_flash')
        self.tune([50, 80, 40, 70]) # Valores por defecto

    def tune(self, params):
        conf_media_fin, conf_alta_inicio, luz_media_fin, luz_clara_inicio = params
        self.confianza_ocr['baja'] = fuzz.trimf(self.confianza_ocr.universe, [0, 0, conf_media_fin])
        self.confianza_ocr['media'] = fuzz.trimf(self.confianza_ocr.universe, [0, conf_media_fin, conf_alta_inicio])
        self.confianza_ocr['alta'] = fuzz.trimf(self.confianza_ocr.universe, [conf_alta_inicio, 100, 100])
        self.luz_ambiental['oscura'] = fuzz.trimf(self.luz_ambiental.universe, [0, 0, luz_media_fin])
        self.luz_ambiental['media'] = fuzz.trimf(self.luz_ambiental.universe, [0, luz_media_fin, luz_clara_inicio])
        self.luz_ambiental['clara'] = fuzz.trimf(self.luz_ambiental.universe, [luz_clara_inicio, 100, 100])
        self.potencia_flash['baja'] = fuzz.trimf(self.potencia_flash.universe, [0, 0, 85])
        self.potencia_flash['media'] = fuzz.trimf(self.potencia_flash.universe, [80, 170, 255])
        self.potencia_flash['alta'] = fuzz.trimf(self.potencia_flash.universe, [200, 255, 255])
        
        rule1 = ctrl.Rule(self.confianza_ocr['baja'], self.potencia_flash['alta'])
        rule2 = ctrl.Rule(self.confianza_ocr['media'] & self.luz_ambiental['oscura'], self.potencia_flash['media'])
        rule3 = ctrl.Rule(self.confianza_ocr['alta'] | self.luz_ambiental['clara'], self.potencia_flash['baja'])
        
        self.control_system = ctrl.ControlSystem([rule1, rule2, rule3])
        self.simulation = ctrl.ControlSystemSimulation(self.control_system)

    def compute(self, input_luz, input_confianza):
        self.simulation.input['luz_ambiental'] = input_luz
        self.simulation.input['confianza_ocr'] = input_confianza
        try:
            self.simulation.compute()
            return self.simulation.output['potencia_flash']
        except:
            return 50.0

# --- FUNCIONES DE HARDWARE Y SIMULACIÓN ---
def iniciar_simulador_visual():
    global dashboard
    dashboard = np.zeros((600, 800, 3), dtype="uint8")
    cv2.namedWindow("Simulador de Hardware Arduino (OpenCV)")
    cv2.imshow("Simulador de Hardware Arduino (OpenCV)", dashboard)
    print("✅ Modo simulación visual con OpenCV activado.")

def conectar_arduino(port='/dev/ttyACM0', baudrate=9600):
    global SIMULATION_MODE
    try:
        # Pyserial puede no estar instalado en todos los entornos, lo importamos aquí
        import serial
        arduino = serial.Serial(port, baudrate, timeout=1)
        print(f"✅ Conexión con Arduino real establecida en {port}.")
        SIMULATION_MODE = False
        return arduino
    except (ImportError, serial.SerialException, NameError):
        print(f"⚠️  No se encontró Arduino o PySerial. Iniciando simulación visual.")
        SIMULATION_MODE = True
        iniciar_simulador_visual()
        return {"status": "simulado"}

def enviar_comando_arduino(arduino, comando):
    global target_angle, flash_brightness, log_messages
    if not SIMULATION_MODE:
        if arduino and arduino.is_open:
            arduino.write(f"{comando}\\n".encode())
    else:
        log_messages.append(f"➡️  Comando recibido: {comando}")
        if comando == "OPEN":
            target_angle = -90
        elif comando.startswith("SET_FLASH:"):
            try:
                power = int(comando.split(":")[1])
                flash_brightness = power
            except:
                flash_brightness = 0

def actualizar_y_manejar_eventos_simulador(live_frame=None):
    global barrier_angle, target_angle, flash_brightness, log_messages, dashboard
    if not SIMULATION_MODE:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            return False
        return True

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or cv2.getWindowProperty("Simulador de Hardware Arduino (OpenCV)", cv2.WND_PROP_VISIBLE) < 1:
        return False

    # Dibujar el dashboard
    if live_frame is not None:
        dashboard = cv2.resize(live_frame, (800, 600))
    else:
        dashboard.fill(50)
    
    # Animar y dibujar componentes
    if barrier_angle > target_angle: barrier_angle -= 3
    if barrier_angle < -90: target_angle = 0 # Cerrar automáticamente
    rad = np.deg2rad(barrier_angle)
    end_x = int(400 + 350 * np.cos(rad))
    end_y = int(525 + 350 * np.sin(rad))
    cv2.rectangle(dashboard, (375, 500), (425, 550), (0, 0, 0), -1)
    cv2.line(dashboard, (400, 525), (end_x, end_y), (80, 80, 255), 15)
    flash_color = (0, flash_brightness, flash_brightness)
    cv2.circle(dashboard, (50, 50), 30, flash_color, -1)
    for i, msg in enumerate(log_messages[-15:]):
        cv2.putText(dashboard, msg, (20, 20 + i * 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
    cv2.imshow("Simulador de Hardware Arduino (OpenCV)", dashboard)
    return True

def cerrar_conexion_arduino(arduino):
    if not SIMULATION_MODE:
        if arduino and 'is_open' in dir(arduino) and arduino.is_open:
            arduino.close()
    else:
        cv2.destroyAllWindows()
"""
with open("arduino_python.py", "w") as f:
    f.write(arduino_python_code)
print("Generado: arduino_python.py")


# --- Contenido de main.py ---
main_code = """
import cv2
import time
import numpy as np
import os
from ultralytics import YOLO
from stable_baselines3 import PPO

# Importaciones de nuestros módulos
from arduino_python import FuzzyController, conectar_arduino, enviar_comando_arduino, cerrar_conexion_arduino, actualizar_y_manejar_eventos_simulador, log_messages
from database_sql import conectar_sql_server, guardar_en_base_de_datos, verificar_placa_registrada
from preprocesamiento import preprocesar_placa, detectar_texto

# --- 1. SETUP ---
print("⚙️  Iniciando sistema de ALPR...")
try:
    yolo_model = YOLO("runs/detect/placas_v14/weights/best.pt")
    agente_ppo = PPO.load("sistema_control_adaptativo/agentes_entrenados/ppo_fuzzy_ocr_controller.zip")
    fuzzy_controller = FuzzyController()
    print("✅ Modelos de IA cargados.")
except Exception as e:
    print(f"❌ Error al cargar modelos de IA: {e}")
    exit()

conn = conectar_sql_server()
# Esta función ahora inicia el Arduino real O la simulación visual
arduino = conectar_arduino()

if conn is None or arduino is None:
    print("❌ Finalizando por error de conexión.")
    exit()

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Error: No se pudo abrir la cámara.")
    exit()

# --- 2. BUCLE PRINCIPAL ---
running = True
is_processing = False
print("🚀 Sistema listo. Presiona 'D' para detectar o 'Q' para salir.")

while running:
    ret, frame = cap.read()
    if not ret:
        time.sleep(0.1)
        continue

    # Esta función dibuja el simulador (si está activo) y maneja la tecla 'q' para salir
    running = actualizar_y_manejar_eventos_simulador(frame)
    if not running:
        break
    
    # La lógica de control se activa si no está ocupada y se presiona 'd' en modo simulación
    # o simplemente si se detecta una placa en modo real
    should_start_processing = False
    if not is_processing:
        key = cv2.waitKey(1) & 0xFF
        # En modo simulación, la 'd' es el disparador
        if 'SIMULATION_MODE' in globals() and SIMULATION_MODE and key == ord('d'):
             should_start_processing = True
        # En modo real, la detección de YOLO es el disparador
        elif 'SIMULATION_MODE' in globals() and not SIMULATION_MODE:
            results = yolo_model(frame, imgsz=320, conf=0.5, verbose=False)
            if results and results[0].boxes:
                should_start_processing = True

    if should_start_processing:
        is_processing = True
        log_messages[:] = ["🚗 Vehículo detectado! Iniciando ciclo..."]
        
        ambient_light = np.mean(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        observation = np.array([ambient_light / 2.55, 0], dtype=np.float32)

        action, _ = agente_ppo.predict(observation, deterministic=True)
        fuzzy_controller.tune(action)
        flash_power = fuzzy_controller.compute(observation[0], 0)
        
        log_messages.append(f"💡 Flash calculado: {int(flash_power)}")
        enviar_comando_arduino(arduino, f"SET_FLASH:{int(flash_power)}")
        time.sleep(0.5)
        
        ret_final, final_frame = cap.read()
        enviar_comando_arduino(arduino, "SET_FLASH:0")

        if ret_final:
            final_results = yolo_model(final_frame, imgsz=320, conf=0.5, verbose=False)
            if final_results and final_results[0].boxes:
                box = final_results[0].boxes[0]
                roi = final_frame[int(box.xyxy[0][1]):int(box.xyxy[0][3]), int(box.xyxy[0][0]):int(box.xyxy[0][2])]
                placa_texto, confianza = detectar_texto(preprocesar_placa(roi))

                if placa_texto:
                    log_messages.append(f"✅ Placa: {placa_texto} (Conf: {confianza:.1f}%)")
                    if verificar_placa_registrada(conn, placa_texto):
                        log_messages.append("➡️ AUTORIZADO. Abriendo barrera.")
                        enviar_comando_arduino(arduino, "OPEN")
                    else:
                        log_messages.append("➡️ NO REGISTRADO.")
                else:
                    log_messages.append("❌ OCR falló.")
            else:
                log_messages.append("❌ No se detectó placa en foto final.")
        
        log_messages.append("---------------------------------")
        log_messages.append("Esperando siguiente detección...")
        time.sleep(5)
        is_processing = False

# --- 3. LIMPIEZA ---
print("Cerrando sistema...")
cap.release()
cerrar_conexion_arduino(arduino)
if conn:
    conn.close()
"""
with open("main.py", "w") as f:
    f.write(main_code)
print("Generado: main.py")

# ... (Generadores para los otros archivos: preprocesamiento, database_sql, simulation_manager, ocr_environment, train_agent) ...
# ... Estos no necesitan cambios, pero los incluimos para tener una base de código completa y verificada ...

preprocesamiento_code = """
import cv2
import pytesseract
import pandas as pd
import re

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

def preprocesar_placa(roi):
    try:
        gris = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gauss = cv2.GaussianBlur(gris, (5, 5), 0)
        umbral = cv2.adaptiveThreshold(
            gauss, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
        )
        return umbral
    except Exception:
        return None

def detectar_texto(img_preprocesada):
    try:
        if img_preprocesada is None: return None, 0
        config = "--psm 8 --oem 3"
        data = pytesseract.image_to_data(img_preprocesada, config=config, output_type=pytesseract.Output.DATAFRAME)
        data = data[data.conf > 0]
        
        if not data.empty:
            texto_detectado = "".join(data['text'].astype(str))
            texto_normalizado = re.sub(r'[^A-Z0-9]', '', texto_detectado.upper())
            confianza_promedio = data['conf'].mean()
            return texto_normalizado, confianza_promedio
        else:
            return None, 0
    except Exception:
        return None, 0
"""
with open("preprocesamiento.py", "w") as f:
    f.write(preprocesamiento_code)
print("Generado: preprocesamiento.py")

database_code = """
import pyodbc
import tkinter as tk
from tkinter import messagebox

def conectar_sql_server():
    try:
        connection_string = (
            "Driver={ODBC Driver 18 for SQL Server};"
            "Server=tcp:servidor-alpr-test-final.database.windows.net,1433;"
            "Database=Registro_Placas;"
            "Uid=jhoan;"
            "Pwd=Capston_2025!;"
            "Encrypt=yes;"
            "TrustServerCertificate=no;"
            "Connection Timeout=30;"
        )
        conn = pyodbc.connect(connection_string)
        print("✅ Conexión con Azure SQL Database establecida.")
        return conn
    except Exception as e:
        print(f"❌ Error al conectar con Azure SQL Database: {e}")
        return None

def guardar_en_base_de_datos(conn, placa):
    # El resto de las funciones de DB no necesitan cambios
    try:
        cursor = conn.cursor()
        query = "INSERT INTO PlacasDetectadas (Placa) VALUES (?)"
        cursor.execute(query, placa)
        conn.commit()
    except Exception: pass

def verificar_placa_registrada(conn, placa):
    try:
        cursor = conn.cursor()
        query = "SELECT * FROM VehiculosRegistrados WHERE Placa = ?"
        cursor.execute(query, placa)
        return cursor.fetchone()
    except Exception:
        return None
"""
with open("database_sql.py", "w") as f:
    f.write(database_code)
print("Generado: database_sql.py")

simulation_manager_code = """
import cv2
import numpy as np
import os
import random

class SimulationManager:
    def __init__(self, environments_base_path):
        self.base_path = environments_base_path
        self.available_environments = [d for d in os.listdir(self.base_path) 
                                       if os.path.isdir(os.path.join(self.base_path, d))]
        if not self.available_environments:
            raise FileNotFoundError(f"No se encontraron directorios de entornos en '{self.base_path}'")
        self.current_env_name = None
        self.sorted_images_by_intensity = []

    def load_environment(self, env_name):
        if env_name not in self.available_environments:
            raise ValueError(f"El entorno '{env_name}' no existe.")
        self.current_env_name = env_name
        env_path = os.path.join(self.base_path, env_name)
        images = {}
        for filename in os.listdir(env_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                path = os.path.join(env_path, filename)
                img = cv2.imread(path)
                if img is not None: images[filename] = img
        if len(images) < 2:
            raise ValueError(f"El entorno '{env_name}' debe tener al menos 2 imágenes.")
        self.sorted_images_by_intensity = sorted(images.values(), key=lambda img: np.mean(cv2.cvtColor(img, cv2.COLOR_BGR2HSV)[:, :, 2]))

    def get_random_environment_name(self):
        return random.choice(self.available_environments)

    def simulate_lighting(self, intensity, activation_threshold=0.25):
        if not self.current_env_name: raise RuntimeError("Carga un entorno primero.")
        intensity = np.clip(float(intensity), 0.0, 1.0)
        darkest_img = self.sorted_images_by_intensity[0]
        headlight_img = self.sorted_images_by_intensity[1]
        if intensity <= activation_threshold:
            base_dinamica = darkest_img.astype(np.float32)
        else:
            mix_factor = (intensity - activation_threshold) / (1.0 - activation_threshold)
            base_dinamica = cv2.addWeighted(darkest_img, 1.0 - mix_factor, headlight_img, mix_factor, 0).astype(np.float32)
        beta = (intensity - 0.5) * 50
        alpha = 1.0 + (intensity * 0.2)
        return cv2.convertScaleAbs(base_dinamica, alpha=alpha, beta=beta)
"""
with open("simulation_manager.py", "w") as f:
    f.write(simulation_manager_code)
print("Generado: simulation_manager.py")


ocr_environment_code = """
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from simulation_manager import SimulationManager
from arduino_python import FuzzyController 
from preprocesamiento import preprocesar_placa, detectar_texto
from ultralytics import YOLO

class OCREnvironment(gym.Env):
    def __init__(self):
        super(OCREnvironment, self).__init__()
        self.sim_manager = SimulationManager(environments_base_path='sistema_control_adaptativo/entornos/')
        self.fuzzy_controller = FuzzyController()
        self.yolo_model = YOLO("runs/detect/placas_v14/weights/best.pt")
        self.action_space = spaces.Box(low=np.array([10, 51, 10, 51]), high=np.array([50, 90, 50, 90]), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=100, shape=(2,), dtype=np.float32)

    def step(self, action):
        self.fuzzy_controller.tune(action)
        ambient_light = np.random.uniform(5, 70)
        initial_confidence = 10 
        flash_power = self.fuzzy_controller.compute(ambient_light, initial_confidence)
        intensity = flash_power / 255.0
        simulated_image = self.sim_manager.simulate_lighting(intensity)
        
        ocr_results = self.yolo_model(simulated_image, imgsz=320, conf=0.5, verbose=False)
        final_confidence = 0.0
        if ocr_results and len(ocr_results[0].boxes) > 0:
            box = ocr_results[0].boxes[0]
            roi = simulated_image[int(box.xyxy[0][1]):int(box.xyxy[0][3]), int(box.xyxy[0][0]):int(box.xyxy[0][2])]
            img_preprocesada = preprocesar_placa(roi)
            if img_preprocesada is not None:
                _, confidence = detectar_texto(img_preprocesada)
                final_confidence = confidence if confidence is not None else 0.0

        reward = final_confidence
        terminated = True
        truncated = False
        observation = np.array([ambient_light, final_confidence], dtype=np.float32)
        info = {}
        return observation, reward, terminated, truncated, info

    def reset(self, seed=None):
        if seed is not None:
            super().reset(seed=seed)
        random_env = self.sim_manager.get_random_environment_name()
        self.sim_manager.load_environment(random_env)
        observation = np.array([50, 0], dtype=np.float32)
        info = {}
        return observation, info
"""
with open("ocr_environment.py", "w") as f:
    f.write(ocr_environment_code)
print("Generado: ocr_environment.py")

train_agent_code = """
from stable_baselines3 import PPO
from ocr_environment import OCREnvironment
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

env = OCREnvironment()
model = PPO(
    "MlpPolicy", 
    env, 
    verbose=1, 
    tensorboard_log="sistema_control_adaptativo/logs_tensorboard/", 
    device=device
)

print("\\n🚀 Iniciando entrenamiento del agente PPO...")
model.learn(total_timesteps=10000)
print("\\n✅ Entrenamiento finalizado.")

model.save("sistema_control_adaptativo/agentes_entrenados/ppo_fuzzy_ocr_controller")
print("\\n💾 Agente guardado en 'sistema_control_adaptativo/agentes_entrenados/'")
"""
with open("desarrollo_y_extras/train_agent.py", "w") as f:
    f.write(train_agent_code)
print("Generado: desarrollo_y_extras/train_agent.py")


print("\n--- ¡Proceso completado! Todos los archivos .py esenciales han sido generados/actualizados. ---")

Generando la versión final de todos los archivos del proyecto...
Generado: arduino_python.py
Generado: main.py
Generado: preprocesamiento.py
Generado: database_sql.py
Generado: simulation_manager.py
Generado: ocr_environment.py
Generado: desarrollo_y_extras/train_agent.py

--- ¡Proceso completado! Todos los archivos .py esenciales han sido generados/actualizados. ---
